In [1]:
!export DISABLE_TELEMETRY=YES

In [2]:
import ollama
import pandas as pd
import json

from tqdm import tqdm
#from tqdm.auto import tqdm  # for notebooks

# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

In [3]:
df = pd.read_csv('top10k.csv')

In [4]:
df

,Unnamed: 0,title,selftext,ups
0,74233,TIFU by having my reddit history revealed by J...,This happened to me yesterday and I'm still ba...,85249
1,76661,TIFU by cumming into a coconut,EDIT: I got an AMA thread now. Help me: \n\nht...,81547
2,42440,TIFU by stuffing my face with edibles before d...,"Recently, I traveled to Denver, Colorado with ...",80916
3,35184,TIFU by getting Reddit banned in Russia,"Today Reddit was blocked in Russia, and I am t...",64262
4,73236,TIFU by not telling my wife our son was coming...,This happened Sunday night.\n\n\n\nMy oldest s...,49877
...,...,...,...,...
9995,42961,TIFUpdate by making a dumb joke to myself whic...,[Some of you may remember my stupid joke](http...,130
9996,71249,TIFU by pressing the wrong button,"So by some stroke of luck, I was invited to at...",130
9997,73043,TIFU my first edible experience,"Mandatory ""This actually happened about a year...",130
9998,36945,TIFU by holding the door open for an old man a...,I'm still stunned and have no idea what I did ...,130


In [5]:
def check_vibes(storytext):
    try:
        tries = 0
        while tries < 3:
            approvaljson = ollama.generate(model='qwen2.5:14b', 
                                     format='json',
                                 prompt='''You are the world's best content curator for children and you are choosing stories to adapt to short videos for kids, read the following story and decide if it is family friendly pg-13 and would appeal to all ages including kids. Don't approve stories that have any sexual themes or content related to nudity or drugs or any illegal activity. give your decision in the following json format: {"reasoning":"provide a concise reasoning for your descision here", "approval":true/false} .
                                 here's the story: ''' + storytext)
            approvaldict = json.loads(approvaljson['response'])
            if 'approval' in approvaldict.keys():
                if approvaldict['approval'] == True:
                    print(storytext[:1000])
                # if 'reasoning' in approvaldict.keys():
                #     print(approvaldict['reasoning'])
                return approvaldict['approval']
            else:
                tries = tries + 1
        return False
    except:
        return False
    

In [6]:
df['approval'] = df.progress_apply(lambda row: check_vibes(row.selftext), axis = 1)

  0%|                                                | 21/10000 [01:33<7:17:52,  2.63s/it]

In 5th grade, I had a black history month project where I had to make a presentation on a person of my choice. I looked up ‘important African Americans’ and found a Wikipedia page with a list of famous black people, or so I thought. I chose Allan McLeod Cormack, a South-African American scientist, seeing that he invented the CT scan. I put my blood sweat and tears into that project. Two days before the project was due I decided to put a picture of Cormack on my poster. To my confusion, all I could find was a picture of this white guy. Turns out, he was a white South African. I made the mistake of telling my teacher instead of keeping it a secret, thinking that it would be better to come clean. She let me go through with the presentation portion of the project, but I had to completely start over on my research and redo the write-up on someone actually worthy of black history month. 

TL;DR I did my black history project on a white guy and didn’t realize until the day before it was due, 

  0%|▏                                               | 33/10000 [02:01<6:09:48,  2.23s/it]

This literally happened about 15 minutes ago. I was walking behind this lady pushing a pram with a kid inside playing with a helium  balloon. I suddenly see her baby accidentally let go of his balloon so I instantly turn on my hero-mode and extend my hand to grab the balloon string. It was a really epic and glorious moment and in my mind I was already playing out how cool I looked and how the lady would thank me for saving the balloon. Unfortunately, I did not manage to grab it in time. I did however end up with my hand in a closed fist straight into another man's cheek who was walking next to me. Now there's a crying baby in front of me and a very shocked, confused and upset stranger beside me. I think after the initial shock he did realise I was only trying to do a good deed so he didn't say anything to me but either way I immediately slowed down in embarrassment  and let them walk ahead of me...I am the hero this city does not need.

TL;DR Tried to save a baby's balloon from flying 

  0%|▏                                               | 37/10000 [02:11<6:26:04,  2.33s/it]

...

100%|████████████████████████████████████████████▊| 9967/10000 [11:26:46<01:48,  3.29s/it]

**UPDATE 12/18/2015** - This is after my "final" update - I don't think we are even close the final update yet.  GM is persistent in diagnosing the problem.  Even though we were reluctant to, we agreed to have them take the truck for several weeks to try to diagnose the problem in their controlled settings.  For compensation, they are giving us a brand new 2016 truck as a rental, paying one month of our car payment and giving us an unspecifired OnStar benefit.

Talk about great customer service.  I don't care what the outcome is, it's quite clear we are dealing with a top-notch organization and I think everyone should know that.





We bought a new truck and it came with OnStar.  OnStar has an app that you can start your car from your mobile phone.

We work together.  He gets there four hours before my shift so I always bring him fresh coffee.  Today I forgot the coffee on the kitchen counter.

He was bummed.  We started teaching our senior exercise class and he jokingly made fun of m

100%|████████████████████████████████████████████▉| 9984/10000 [11:27:35<00:44,  2.79s/it]

At work today we ran out of potatoes. My boss asked me if I had a vehicle; I told him I did, so he gave me forty dollars and asked me to run down to the store and buy 80 count potatoes. Alright, I can do that. He said they should be "About that big, nothing too huge", and he said if I could get bags that would be ideal; they would be cheaper that way. So I go to the store and I'm looking at all of the potato options they have.

I see individually wrapped potatoes, just basic potatoes, little potatoes, huge potatoes, and bags of potatoes that look promising. I count about 10 potatoes per bag, and it looks like 40 dollars will get me about 8 bags of them. They're a bit bigger than what I'm shooting for, but it looks like my options are "Spuds the Size of Your Ex's Soul", "Large'ish Potatoes", or "Gargantuan Titan Taters."
 
So I get 8 bags of the big potatoes. I lug them all back to my car and pile them in the passenger seat, and I drive this weird potato-person back to work and take the

100%|████████████████████████████████████████████| 10000/10000 [11:28:38<00:00,  4.13s/it]


In [10]:
df['approval'].describe()

count      441
unique       1
top       True
freq       441
Name: approval, dtype: object

In [17]:
#df.to_csv('top10k_approval.csv')

In [8]:
df = df[df.approval == True]

In [23]:
pd.set_option('display.max_colwidth', None)

In [24]:
df.selftext.head(10)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [9]:
df.to_csv('approved_stories_qwen.csv')